In [66]:
import os
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import sqlite3
from pathlib import Path
import matplotlib.pyplot as plt

In [6]:
chemin_bdd = Path('db') / 'olympics.sqlite3'
conn = sqlite3.connect(chemin_bdd)

# P3. Exploration SQL

Le répertoire `/db` contient une base de données SQLite3 `olympics.sqbpro` reprenant les résultats des épreuves des jeux d'été pour une période donnée (cf P1 & P2).

On s'intéresse dans le cadre de cette partie P3 à l'exploration via SQL des résultats des JO d'été (table : `summer`).

---

### Question 1: Qu'est ce que SQL ?

Le SQL **(Structured Query Language)** est un langage permettant de communiquer avec une base de données.
Ce langage informatique est notamment très utilisé par les développeurs web pour communiquer avec les données d’un site web. Il est possible de créer ou supprimer des objets dans la base de données, mais aussi manipuler des bases de données en effectuant des requêtes qui permettent d'analyser les données de celle-ci.

Ce langage est structuré, c’est-à-dire que la syntaxe est toujours la même et respecte des normes très précises.

### Question 2: Qu'est ce que SQLite 3 ? Avantages et/ou Inconvénients ?

**SQLite** est un système de base de données simplifié de MySQL. SQLite3 est la 3ème version qui apporte de meilleures
performances que les versions précédentes, telles que l'auto increment, un stockage plus efficace et un support des clés étrangères.
Les principaux avantages de SQLite est qu'il est plus simple d'utilisation, fonctionne sans serveur, et possède la majeure partie des fonctions classiques.

Il manque en revanche certaines fonctions qui peuvent s'avérer utiles dans certains cas (ex : ALTER TABLE, COUNT (DISTINCT X). Il ne reconnaît par ailleurs ni les transactions imbriquées, ni les déclencheurs, et ne permet pas la réplication. Il est donc réservé à des besoins basiques.

### Question 3: Donner des exemples d'autres formats de fichier pouvant être utilisé pour fournir des données similaires 

.xls, json

### Question 4: Quel est le type (texte ou binaire) du fichier base de données SQLite3?

Le fichier `olympics.sqbpro` est un fichier binaire.

### Question 5: Donner les caractéristiques de la table `summer` ? 

Cette table donne tous les médaillés des Jeux olympiques d'été, en détaillant pour chaque médaille remportée :

* Année de l'édition

* Ville de l'édition

* Catégorie du sport

* Sport pratiqué

* Nom de l'athlète

* Nationalité de l'athlète

* Genre de l'athlète

* Nom de l'épreuve

* Médaille (or, argent, bronze)

---
### Question 6: Combien d'enregistrements possède la table `summer` ?

In [54]:
query6 = """SELECT count(*) AS "Nb enregistrements"

FROM summer"""

results6 = pd.read_sql_query(query6, conn)
results6

,Nb enregistrements
0,31165


### Question 7: Quelle période des JO couvre ce fichier de résultats ?

In [53]:
query7 = """SELECT min(Year), max(Year)

FROM summer"""

results7 = pd.read_sql_query(query7, conn)
results7

,min(Year),max(Year)
0,1896,2012


### Question 8: Indiquer les années et villes des olympiades ? 

In [13]:
query_8 = """SELECT DISTINCT Year, City

FROM summer"""

results8 = pd.read_sql_query(query_8, conn)
results8

,Year,City
0,1896,Athens
1,1900,Paris
2,1904,St Louis
3,1908,London
4,1912,Stockholm
5,1920,Antwerp
6,1924,Paris
7,1928,Amsterdam
8,1932,Los Angeles
9,1936,Berlin


### Questions Google Drive: 

**Q3. En 2012 combien de médailles le Japon a t-il gagné ?**

In [15]:
query_3 = """SELECT count(*) AS "Nb Médailles"

FROM (

SELECT *

FROM summer

WHERE Year = 2012 AND Country = "JPN"

GROUP BY Sport, Discipline, Gender, Event

)"""

results3 = pd.read_sql_query(query_3, conn)
results3

,Nb Médailles
0,38


**Q17. Entre 1984 et 2012 combien de médailles a gagné la France ?**

In [17]:
query_17 = """SELECT count(*) AS "Nb médailles"

FROM(

SELECT *

FROM summer

WHERE Year >= 1984 AND Year <= 2012 AND Country = "FRA"

GROUP BY Year, Sport, Discipline, Gender, Event

)"""

results17 = pd.read_sql_query(query_17, conn)
results17

,Nb médailles
0,245


**Q10. Combien de joueuses comptait la sélection brésilienne de volley-ball médaillée d’or aux J.O de 2012 ?**

In [21]:
query_10 = """SELECT count(*) AS "Nb joueuses"

FROM summer

WHERE Country = "BRA" AND Discipline = "Volleyball" AND Gender = "Women" AND Year = 2012"""

results10 = pd.read_sql_query(query_10, conn)
results10

,Nb joueuses
0,12


**Q13. Est-ce que la France à déjà gagné une médaille d’or au cricket ?**

In [22]:
query_13 = """SELECT count(*) AS "Nb médailles"

FROM summer

WHERE Country = "FRA" AND Discipline = "Cricket" AND Medal = "Gold" """

results13 = pd.read_sql_query(query_13, conn)
results13

,Nb médailles
0,0


**Q12. Quel est l’athlète qui a remporté le plus de médailles d’or ?**

In [25]:
query_12 = """SELECT Athlete, count(*) AS "Nb médailles"

FROM summer

WHERE Medal = "Gold"

GROUP BY Athlete

ORDER BY COUNT(*) DESC

LIMIT 1"""

results12 = pd.read_sql_query(query_12, conn)
results12

,Athlete,Nb médailles
0,"PHELPS, Michael",18


**Q5. Quel pays a remporté le plus de médailles d’or ?**

In [27]:
query_5 = """SELECT Country, count(*) AS "Nb médailles"

FROM(

SELECT *

FROM summer

WHERE Medal = "Gold"

GROUP BY Year, Sport, Discipline, Gender, Event

ORDER BY Count(*) DESC

)

GROUP BY Country

ORDER BY count(*) DESC

LIMIT 1"""

results5 = pd.read_sql_query(query_5, conn)
results5

,Country,Nb médailles
0,USA,976


**Q15. Le nombre de médailles d’or par pays en 2012**

In [38]:
query_15 = """SELECT Country, count(*) AS "Nb médailles"

FROM(

SELECT *, count(*)

FROM summer

WHERE Medal = "Gold" AND Year = 2012

GROUP BY Sport, Discipline, Event, Gender

)

GROUP BY Country

ORDER BY count(*) DESC"""

results15 = pd.read_sql_query(query_15, conn)
results15

,Country,Nb médailles
0,USA,46
1,CHN,37
2,GBR,30
3,RUS,21
4,KOR,13
5,GER,12
6,FRA,11
7,ITA,8
8,HUN,8
9,AUS,8


**Q1. Combien la France a remporté de médailles d’or aux JO de 1984 ?**

In [41]:
#requête imbriquée#*

query_1 = """SELECT Count(*) AS "Nb médailles"

FROM(

SELECT Athlete, Sport, Discipline, Event, Gender, count(*)

FROM summer

WHERE Year = 1984 AND Country = "FRA"  AND Medal = "Gold"

GROUP BY Sport, Discipline, Event, Gender

)"""

results1 = pd.read_sql_query(query_1, conn)
results1

,Nb médailles
0,5


**Q11. Combien de femmes françaises ont ont remporté des médailles en 2012, quelle sont leurs médailles ?**

In [43]:
query_11a = """SELECT count(DISTINCT Athlete)

FROM summer

WHERE Gender = "Women" AND Country = "FRA" AND Year = "2012" """

results11a = pd.read_sql_query(query_11a, conn)
results11a

,count(DISTINCT Athlete)
0,29


In [44]:
query_11b = """SELECT Medal, count(*)

FROM summer

WHERE Gender = "Women" AND Country = "FRA" AND Year = "2012"

GROUP BY Medal """

results11b = pd.read_sql_query(query_11b, conn)
results11b

,Medal,count(*)
0,Bronze,12
1,Gold,4
2,Silver,15


**Q9. Combien de fois Londres a accueilli les J.O ?**

In [46]:
query_9 = """SELECT count(DISTINCT Year)

FROM summer

WHERE City = "London" """

results9 = pd.read_sql_query(query_9, conn)
results9

,count(DISTINCT Year)
0,3


**Q6. Combien de villes différentes ont accueilli les JO d’été entre 1896 et 2012 ?**

In [55]:
query_6 = """SELECT count(DISTINCT City) AS "Nb villes"

FROM summer

WHERE Year >=1896 AND Year <= 2012"""

results6 = pd.read_sql_query(query_6, conn)
results6

,Nb villes
0,22


**Q16. Combien de femmes ont remporté des médailles ?**

In [51]:
query_16 = """SELECT count(DISTINCT Athlete) AS "Nb femmes"

FROM summer

WHERE Gender = "Women" """

results16 = pd.read_sql_query(query_16, conn)
results16

,Nb femmes
0,5838


**Q7. Combien de femmes ont obtenu  une médaille d’or ?**

In [58]:
query_7 = """SELECT count(DISTINCT Athlete) AS "Nb femmes"

FROM summer

WHERE Gender = "Women" AND Medal = "Gold" """

results7 = pd.read_sql_query(query_7, conn)
results7

,Nb femmes
0,2045


**Q2. Quel pays a le plus d’athlètes ?**

In [62]:
query_2 = """SELECT Country, count(DISTINCT Athlete) AS "Nb athlètes"

FROM summer

GROUP BY Country

ORDER BY count(DISTINCT Athlete) DESC

LIMIT 1"""

results2 = pd.read_sql_query(query_2, conn)
results2

,Country,Nb athlètes
0,USA,3149


**Q8. Combien de sports différents sont représentés par année ?**

In [64]:
query_8 = """SELECT Year, count(DISTINCT Sport) AS "Nb sports"

FROM summer

GROUP BY Year"""

results8 = pd.read_sql_query(query_8, conn)
results8

,Year,Nb sports
0,1896,9
1,1900,19
2,1904,16
3,1908,22
4,1912,14
5,1920,22
6,1924,17
7,1928,14
8,1932,14
9,1936,19


**Q4. Dans quels différents sports les Grecs ont-ils gagné des médailles ?**

In [65]:
query_4 = """SELECT Sport, count(Medal)

FROM summer

WHERE Country = "GRE"

GROUP BY Sport

ORDER BY count(Medal) DESC"""

results4 = pd.read_sql_query(query_4, conn)
results4

,Sport,count(Medal)
0,Athletics,29
1,Aquatics,22
2,Gymnastics,21
3,Weightlifting,15
4,Shooting,14
5,Sailing,14
6,Wrestling,11
7,Rowing,6
8,Taekwondo,4
9,Fencing,4


--- 

Fin